In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Softmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets.mnist import load_data
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:

(train_x, train_y), (test_x, test_y) = load_data()
train_x.shape

2024-11-13 11:03:42.642129: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-13 11:03:42.651636: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731492222.663615   12639 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731492222.666887   12639 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 11:03:42.678632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


(60000, 28, 28)

In [3]:
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = train_x[..., tf.newaxis].astype("float32")
test_x = test_x[..., tf.newaxis].astype("float32")

In [16]:

model = Sequential([
  Input(shape=(28,28,1)), 
  Flatten(),
  Dense(128, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  Dense(10)
])

In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_x, train_y,epochs=5,batch_size=32)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8733 - loss: 0.4337
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9650 - loss: 0.1200
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9766 - loss: 0.0780
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9819 - loss: 0.0596
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9871 - loss: 0.0439


In [18]:
scores = model.evaluate(test_x, test_y)
scores

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9733 - loss: 0.0932


[0.07819551229476929, 0.9767000079154968]

In [32]:

final_model = Sequential([model, Softmax()])
print(final_model(test_x[:1]))


tf.Tensor(
[[1.8365968e-07 2.9224025e-09 2.0331570e-06 1.5050451e-04 1.5351040e-12
  1.3049449e-08 2.8534580e-13 9.9984360e-01 5.3446890e-08 3.6724371e-06]], shape=(1, 10), dtype=float32)


In [25]:

def create(drop_out):
    model = Sequential([
        Input(shape=(28,28,1)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(drop_out),
        Dense(10), 
        Softmax()
    ])
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
    return model

classifier = KerasClassifier(create, drop_out=0.1)

params = {'drop_out':[0.1, 0.2, 0.3]}
grid = GridSearchCV(classifier, params)
grid.fit(train_x, train_y)

1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8557 - loss: 0.4977
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8575 - loss: 0.4932
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8609 - loss: 0.5000
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8567 - loss: 0.4959
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8562 - loss: 0.4985
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8467 - loss: 0.5158
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8466 - loss: 0.5240
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8518 - loss: 0.5228
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy:

GridSearchCV(estimator=KerasClassifier(drop_out=0.1, model=<function create at 0x744744275120>),
             param_grid={'drop_out': [0.1, 0.2, 0.3]})

In [26]:
results = pd.DataFrame(grid.cv_results_)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_drop_out,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.162730,0.516855,0.742782,0.078080,0.1,{'drop_out': 0.1},0.954167,0.949917,0.949000,0.949250,0.955750,0.951617,0.002790,1
1,4.070818,0.527200,0.693381,0.197113,0.2,{'drop_out': 0.2},0.952833,0.945000,0.949833,0.948667,0.954417,0.950150,0.003293,2
2,4.067456,0.947562,0.601299,0.129343,0.3,{'drop_out': 0.3},0.952750,0.946500,0.945833,0.944333,0.952667,0.948417,0.003574,3


In [30]:
grid.best_estimator_.model_.evaluate(test_x, test_y, batch_size=32)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9501 - loss: 0.1637


[0.14071416854858398, 0.9574000239372253]